In [ ]:
import os
import cv2

In [ ]:
images = []
labels = []

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 58 kB 3.1 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=924d31f84205c5feb3b2536f278820a23d38d5d2ba522bb5ee3573295f4b2513
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "*" # username from the json file
os.environ['KAGGLE_KEY'] = "**" # Provide your key from the json file
# !kaggle competitions download -c fake-image-classification-challenge # api copied from kaggle
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 99% 328M/330M [00:04<00:00, 169MB/s]
100% 330M/330M [00:04<00:00, 84.2MB/s]


In [ ]:
from zipfile import ZipFile

file_name = "/content/face-mask-12k-images-dataset.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
!pip install tensorflow

In [ ]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [ ]:
# face_cascade = cv2.CascadeClassifier('C:/Users/Murtaza Hasan/Desktop/Artificial Intelligence/haarcascade_frontalface_default.xml')
trainX=[]
validationX=[]
trainY=[]
validationY=[]


In [ ]:
for i in ['WithMask', 'WithoutMask']:
    imageNameList=os.listdir("Face Mask Dataset/Train/" + str(i))
    for fileName in imageNameList:
        Image=cv2.imread("Face Mask Dataset/Train/" + str(i) + "/" + str(fileName))
        try:

            Image=cv2.resize(Image,(64,64))
            trainX.append(Image)
            trainY.append(i)

        except:
            pass
    print("Inside folder ", i)

Inside folder  WithMask
Inside folder  WithoutMask


In [ ]:
for i in ['WithMask', 'WithoutMask']:
    imageNameList=os.listdir("Face Mask Dataset/Validation/" + str(i))
    for fileName in imageNameList:
        Image=cv2.imread("Face Mask Dataset/Validation/" + str(i) + "/" + str(fileName))
        try:

            Image=cv2.resize(Image,(64,64))
            validationX.append(Image)
            validationY.append(i)

        except:
            pass
    print("Inside folder ", i)

Inside folder  WithMask
Inside folder  WithoutMask


In [ ]:
trainX=np.array(trainX)
trainX=trainX/255
trainY=np.array(trainY)

In [ ]:
validationX=np.array(validationX)
validationX=validationX/255
validationY=np.array(validationY)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainY = le.fit_transform(trainY)
validationY = le.fit_transform(validationY)

In [ ]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

In [ ]:
trainX,validationX,trainY,validationY=train_test_split(images,labels,test_size=0.3,stratify=labels)

In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=(64,64, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                        

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

In [ ]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
313/313 [==============================] - 23s 40ms/step - loss: 1.3399 - accuracy: 0.9657 - val_loss: 1.0653 - val_accuracy: 0.9875
Epoch 2/70
313/313 [==============================] - 12s 37ms/step - loss: 0.8933 - accuracy: 0.9819 - val_loss: 0.7386 - val_accuracy: 0.9862
Epoch 3/70
313/313 [==============================] - 12s 37ms/step - loss: 0.6553 - accuracy: 0.9819 - val_loss: 0.5627 - val_accuracy: 0.9912
Epoch 4/70
313/313 [==============================] - 12s 39ms/step - loss: 0.5047 - accuracy: 0.9841 - val_loss: 0.4251 - val_accuracy: 0.9937
Epoch 5/70
313/313 [==============================] - 12s 38ms/step - loss: 0.3732 - accuracy: 0.9860 - val_loss: 0.3452 - val_accuracy: 0.9887
Epoch 6/70
313/313 [==============================] - 12s 37ms/step - loss: 0.3165 - accuracy: 0.9856 - val_loss: 0.2812 - val_accuracy: 0.9862
Epoch 7/70
313/313 [==============================] - 12s 38ms/step - loss: 0.3227 - accuracy: 0.9801 - val_loss: 0.3007 - val_accuracy:

In [ ]:
model.save('mask_model.h5')